API docs for the WikiSRATMicroService
===

**SRAT code is all in SQL, run from this notebook.**  
This code is within the GitHub repository. The result of this will be a table of NHDplus catchments. 

**Note on `ploadrate_total`**  
The microservice output says "tploadrate_total", however I [Mike] suspect this is a mistake. This is actually the local NHDplus catchment load. The stream concentrations take into account the watershed load, but this value is not returned. I went ahead and calculated this when I export the microservice results to PG for convenience / if we need the value later.

**Instances where load reduction is greater than the load created by an NHDplus catchment:**
- After weekly check in call, decided that it can be negative, so leave as raw value
- Otherwise can set threshold to where can't reduce loads beyond a proportion of the current load

In [1]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import json
import os
import psycopg2
from StringParser import StringParser
from DatabaseAdapter import DatabaseAdapter
from DatabaseFormatter import DatabaseFormatter
from DatabaseMakeTable import DatabaseMakeTable

# Run WikiSRATMicroService API

## Open Connection to WikiSRATMicroService Database

To fetch information to assist in formating requests to the WikiSRATMicroService API.

In [2]:
# GET THE DATABASE CONFIG INFORMATION USING A CONFIG FILE. THE FILE IS IN THE GITIGNORE SO WILL REQUIRE BEING SENT

config_file = json.load(open('db_config.json'))
PG_CONFIG = config_file['PG_CONFIG']

_host = PG_CONFIG['host'],
_database = PG_CONFIG['database'],
_user = PG_CONFIG['user'],
_password = PG_CONFIG['password'],
_port = PG_CONFIG['port']

In [3]:
# Create connection to database

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

## Fetch data for formating requests to the WikiSRATMicroService API


In [4]:
%%time

# GET THE MODELED LOADS FROM THE DRWI DATABASE, DERIVED FROM MMW MODEL RUNS

_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw order by huc12;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
print(len(_dbdata))

484
CPU times: user 12.6 ms, sys: 5.73 ms, total: 18.4 ms
Wall time: 603 ms


In [5]:
_dbdata?

Type:        list
String form: [(1, '020401010101', Decimal('83.25'), Decimal('131587.70'), Decimal('144086.00'), Decimal('11916 <...> 00421087362510914, 0.0325741241610815, 0.124997522620959, 0.149069147756057, 0.0349152439841768)]
Length:      484
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


## Run WikiSRATMicroService API


In [6]:
# CREATE A COUPLE HELPER FUNCTIONS TO RUN THE MICROSERVICE
def respond(err, res=None):
    return {
        'statusCode': '400' if err else '200',
        'body': err.args[0] if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
    }

def lambda_handler(event, context):
    try:
        data = StringParser.parse(event['body'])
        db = DatabaseAdapter(_database[0], _user[0], _host[0], _port, _password[0], _flag)
        input_array = DatabaseAdapter.python_to_array(data)
        return respond(None, db.run_model(input_array))
    except AttributeError as e:
        return respond(e)

In [7]:
%%time

# FOR ALL DRWI HUC12s, FEED THROUGH THE MICROSERVICE TO GET SUB-BASIN ATTENUATION
# The database adapter routine flag can either be 'base' or 'restoration', depending on if you want these
# projects to be removed from the attenuation routine. Restoration projects come from what was enetered in
# through FieldDoc.

_flag = 'base'


# RUN THE HUC12s THROUGH THE MICROSERVICE
_body = DatabaseFormatter.parse(_dbdata)
# _body = '[{"huc12": "020402010101", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010102", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010103", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}]'

_r = dict(lambda_handler({"body": _body},None))

CPU times: user 341 ms, sys: 45.6 ms, total: 387 ms
Wall time: 5.94 s


In [8]:
# Explore the API response
_r?

Type:        dict
String form: {'statusCode': '200', 'body': '{"huc12s": {"020401010101": {"huc12": "020401010101", "tpload_hp": <...> 15247}}}}}', 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}}
Length:      3
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [11]:
# Extract the NHD Loads from the response
_nhdloads = dict(json.loads(_r['body']))['huc12s']
_nhdloads?

Type:        dict
String form: {'020401010101': {'huc12': '020401010101', 'tpload_hp': 602.07665754054, 'tpload_crop': 360.72733 <...> 0436867554431356, 'tssloadrate_total': 25055.0654476434, 'tssloadrate_conc': 41.5994419315247}}}}
Length:      484
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [10]:
# Explore selection of data by HUC12
print(dict(json.loads(_r['body']))['huc12s']['020402010101']['catchments'])

{'4481881': {'comid': 4481881, 'tploadrate_total': 8.67678906666698, 'tploadate_conc': 0.00480867068316327, 'tnloadrate_total': 205.478173246534, 'tnloadrate_conc': 0.188308808608071, 'tssloadrate_total': 12430.9823731758, 'tssloadrate_conc': 10.6329585165398}, '4481681': {'comid': 4481681, 'tploadrate_total': 45.9269178229512, 'tploadate_conc': 0.0139356826495012, 'tnloadrate_total': 1098.12798966065, 'tnloadrate_conc': 0.333206840298741, 'tssloadrate_total': 128512.512354698, 'tssloadrate_conc': 38.9947698116636}, '4481279': {'comid': 4481279, 'tploadrate_total': 9.3369485692889, 'tploadate_conc': 0.0870941956538323, 'tnloadrate_total': 260.499630512306, 'tnloadrate_conc': 2.34191601922763, 'tssloadrate_total': 4135.72100063273, 'tssloadrate_conc': 49.6417992751781}, '4481935': {'comid': 4481935, 'tploadrate_total': 47.8207522003034, 'tploadate_conc': 0.107891339771179, 'tnloadrate_total': 1122.82652578667, 'tnloadrate_conc': 2.53327797292518, 'tssloadrate_total': 29233.9090668937, '

# Load Results into Database

Loading results into the database is a time consuming process (~20-30 minutes), so it only needs to be run once for every data update. 

### If you have done this for the latest data, skip this block.


In [9]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON

t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1
        
t

19496

## Save "Base" Model Results (i.e. no BMPs) to Database

In [5]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
# This uses an imported function to create the table. This is necessary to get the COMID geometries

# SET THE TABLE NAME AND CREATE TABLE
tablename_base = 'base_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_base)
new.make_table()


Table Created


In [6]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE ~10 MINUTES

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_base, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)
print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done


## Repeat for Restoration Results

Reruns the WikiSRATMicroService API with restoration BMPs and saves response to the database.

In [7]:
# NOW RUN THE ATTENUATION WITH THE BMPs ADDED IN
_flag = 'restoration'

_r = dict(lambda_handler({"body": _body},None))
print('done')

done


In [8]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON
_nhdloads = dict(json.loads(_r['body']))['huc12s']
t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1

In [9]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
tablename_rest = 'restoration_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_rest)
new.make_table()

Table Created


In [10]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE ~10 MINUTES

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_rest, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)

print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done


# ABOVE CODE ONLY NEEDS TO BE RUN ONCE

Use labs to get outline to collapse code to be run once. Further edits to the data frames for API output can be done here.

# Read Results in Pandas and Save to Local Storage

In [14]:
# PUT HERE ANY TABLE NAMES CREATED ABOVE, IN CASE THE KERNAL WAS RESTARTED AND YOU DON'T WANT TO RUN THE ABOVE AGAIN
tablename_rest = 'restoration_run'
tablename_base = 'base_run'

## Read from Database into Pandas DataFrame

In [ ]:
%%time

# LOAD THE DATABASE TABLES INTO PANDAS
# geom = NHDplus streamline
# catchment = NHDplus catchment

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    _cur = conn.cursor()
    try:
        _cur.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        _cur.close()
        return 1
    
    # We get a list of tupples
    tupples = _cur.fetchall()
    _cur.close()
    
    # Turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

colnames = ('comid','tploadrate_total','tploadate_conc','tnloadrate_total','tnloadate_conc','tssloadrate_total',
            'tssloadate_conc','catchment_hectares','watershed_hectares','tploadrate_total_ws','tnloadrate_total_ws',
            'tssloadrate_total_ws','maflowv','geom','geom_catchment', 'cluster', 'fa_name', 'sub_focusarea',
            'nord','nordstop')

cluster_names = ('Brandywine and Christina','Kirkwood - Cohansey Aquifer','Middle Schuylkill','New Jersey Highlands',
                 'Poconos and Kittatinny','Schuylkill Highlands','Upper Lehigh','Upstream Suburban Philadelphia')


base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = postgresql_to_dataframe(_PG_Connection, base_model_select, colnames)
rest_df = postgresql_to_dataframe(_PG_Connection, rest_model_select, colnames)
print('done')
base_df


## Fetch Additional Geospatial Data from Database

In [4]:
# import data with geopandas compatible geometry
from sqlalchemy import create_engine  
import geopandas as gpd 

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

tablename_base = 'base_run'
tablename_rest = 'restoration_run'

base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = gpd.read_postgis(base_model_select, con)
rest_df = gpd.read_postgis(rest_model_select, con)

In [5]:
# get catchment geometry
base_catch = 'SELECT geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_catch = 'SELECT geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df_catch = gpd.read_postgis(base_catch, con, geom_col="geom_catchment")
rest_df_catch = gpd.read_postgis(rest_catch, con, geom_col="geom_catchment")

In [6]:
# merge
base_df['geom_catchment'] = base_df_catch['geom_catchment']
rest_df['geom_catchment'] = base_df_catch['geom_catchment']

In [ ]:
base_df

## Save to Parquet File
The code below converts the data into a locally saved parquet file to avoid having to access the database every time we run the visualization script.  

Apache Parquet has become the high-performance binary cloud format of choice for storing dataframes.
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_parquet.html
- https://anaconda.org/TomAugspurger/pandas-performance/notebook

In [7]:
# requires pyarrow
import pyarrow
base_df.to_parquet('data/base_df.parquet')
rest_df.to_parquet('data/rest_df.parquet')

C:\Users\sjordan\AppData\Local\Temp/ipykernel_3740/3355558133.py:3: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  base_df.to_parquet('base_df.parquet')
C:\Users\sjordan\AppData\Local\Temp/ipykernel_3740/3355558133.py:4: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.